In [1]:
import pandas as pd

In [2]:
from data.data import Data

ROWS = 400 # Limit 6000
# PreProcess it ALL!!!!
## SIKE NOT ENOUGH MEMORY
# LIMIT TO 10K ROWS INPUT #

data_raw = Data().get_all_data()


# 8670 ROWS INPUT #
df1 = data_raw['twitter_MBTI']
df1 = df1[['label', 'text']]
df1.rename(columns={'label': 'type'}, inplace=True)
df1 = df1.sample(n = ROWS, ignore_index = True, random_state = 1)

# 106062 ROWS INPUT #
df2 = data_raw['MBTI 500']
df2 = df2[['type', 'posts']]
df2.rename(columns={'posts': 'text'}, inplace=True)
df2 = df2.sample(n = ROWS, ignore_index = True, random_state = 1) # > 15000 combined, > 20000 df1 + df2

## SPECIFCALLY CREATING DIV 0 ERROR, ASK ABOUT IT ##
# 7811 ROWS INPUT #
df3 = data_raw['mbti_1']
df3.rename(columns={'posts': 'text'}, inplace=True)
df3 = df3.sample(n = ROWS, ignore_index = True, random_state = 1)
# ####################################################

## Combined all data for PP if desired
data_combined = pd.concat([df1, df2, df3], ignore_index=True)
data_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


/tmp/ipykernel_1017/3695169631.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'label': 'type'}, inplace=True)


In [3]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize


## PreProccessing Still does not have proper pkl storage -> Check your own file first

df_pp = training_preprocessing(data_combined)
df_pp_os = training_oversampling(df_pp)
df_pp_os_bal = training_balancing(df_pp_os)
df_list = training_vectorize(df_pp_os_bal)


Dataset contains 122553 rows

cleaned dataset contains 1200 rows and 8 columns
E-I dataset contains 545 rows and 2379 columns
S-N dataset contains 291 rows and 2556 columns
F-T dataset contains 1219 rows and 2408 columns
P-J dataset contains 956 rows and 2436 columns


In [4]:
from model.model import *
model = initialize_model()

hist = train_model(df_list, model)

hist

F1-score:53.773
Model Type: ie
F1-score:57.208000000000006
Model Type: ns
F1-score:58.867999999999995
Model Type: ft
F1-score:43.131
Model Type: pj


([SGDClassifier(), SGDClassifier(), SGDClassifier(), SGDClassifier()],
 [{'e': {'precision': 0.6153846153846154,
    'recall': 0.23529411764705882,
    'f1-score': 0.3404255319148936,
    'support': 68},
   'i': {'precision': 0.6231884057971014,
    'recall': 0.8958333333333334,
    'f1-score': 0.735042735042735,
    'support': 96},
   'accuracy': 0.6219512195121951,
   'macro avg': {'precision': 0.6192865105908585,
    'recall': 0.5655637254901961,
    'f1-score': 0.5377341334788143,
    'support': 164},
   'weighted avg': {'precision': 0.6199526878211926,
    'recall': 0.6219512195121951,
    'f1-score': 0.5714209678921666,
    'support': 164},
   'bal_acc': 0.6192865105908585},
  {'n': {'precision': 0.7230769230769231,
    'recall': 0.7833333333333333,
    'f1-score': 0.752,
    'support': 60},
   's': {'precision': 0.43478260869565216,
    'recall': 0.35714285714285715,
    'f1-score': 0.39215686274509803,
    'support': 28},
   'accuracy': 0.6477272727272727,
   'macro avg': {'pre

In [5]:
# ENTP #

df = data_raw['MBTI 500']
df = df[['type', 'posts']]
df.rename(columns={'posts': 'text'}, inplace=True)
df = df.sample(n = 10, ignore_index = True, random_state = 1)

df_pred = df.iloc[:1].drop(columns =['type'])
df_pred

,text
0,put honestly look bryan look like guy passion ...


In [6]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize

## Current Step for Work, error is lack of terms?

df_pred_pp = prediction_preprocessing(df_pred)
df_pred_list = prediction_vectorize(df_pred_pp)

In [7]:
df_pred_list[0]

,avg_word_length,type_to_token_ratio,aa,ab,ability,able,abortion,absolute,absolutely,abt,...,younger,youre,youtube,youu,youve,yr,yung,yup,zero,zone
0,5.339394,0.656504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Making sure to run the 4 models with the 4 dataframes, again a pipeline makes more sense
 
# When predicting, the model needs to see the same features as the original fit
# So with new text, how do we pass the new features? 
# Would uploading a pickle file help?
 
predict_model([model,model,model,model], df_pred_list)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ada
- aespa
- alex
- announce
- april
- ...
Feature names seen at fit time, yet now missing:
- abstract
- accomplish
- acknowledge
- acquaintance
- actively
- ...
